# ARCHS4 module-trait association analysis

In [1]:
library(here)
library(dplyr)

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [5]:
# archs4_traits  <- readRDS(here('data/archs4/traits/archs4-phenoplier.rds'))
archs4_plier2 <- readRDS(here('output/archs4/archs4_PLIER2.rds'))

In [6]:
archs4_z <- archs4_plier2$Z
head(archs4_z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV1208,LV1209,LV1210,LV1211,LV1212,LV1213,LV1214,LV1215,LV1216,LV1217
TSPAN6,0,0,0,0.000000,0.0000000,0.0000000,0,0.9190236,0,0,⋯,0,0.667423,0,0,0.0000000,0.9545316,0.0000000,0,0,0.3907894
TNMD,0,0,0,1.118337,0.0000000,0.0000000,0,1.2158565,0,0,⋯,0,0.000000,0,0,0.0000000,1.0771154,0.0000000,0,0,0.0000000
DPM1,0,0,0,0.000000,0.8696805,0.0000000,0,0.0000000,0,0,⋯,0,0.000000,0,0,0.0000000,0.0000000,0.0000000,0,0,0.6387958
SCYL3,0,0,0,0.000000,0.0000000,0.0000000,0,0.0000000,0,0,⋯,0,0.000000,0,0,0.0000000,0.0000000,0.8014029,0,0,0.0000000
C1orf112,0,0,0,0.000000,0.0000000,0.0000000,0,0.3201366,0,0,⋯,0,0.000000,0,0,0.4986467,0.0000000,1.4541659,0,0,0.0000000
FGR,0,0,0,0.000000,0.0000000,0.5699193,0,0.0000000,0,0,⋯,0,0.000000,0,0,0.4770875,0.0000000,1.4511744,0,0,1.0046331


In [ ]:
# archs4_traits_sig <- archs4_traits  %>% 
# dplyr::filter(fdr < 0.05)  %>% 
# dplyr::select(phenotype_desc, lv) %>%
# dplyr::rename(phenotype = phenotype_desc, LV = lv)
# archs4_traits_sig_collapsed  <- archs4_traits_sig  %>% 
# dplyr::group_by(LV)  %>% 
# dplyr::summarise(phenotype = paste(phenotype, collapse = '; '))
# head(archs4_traits_sig_collapsed)

In [17]:
archs4_plier2_base <- readRDS(here('output/archs4/archs4_baseRes.rds'))

In [16]:
archs4_plier2$summary

pathway,LV,AUC,p-value,FDR
<chr>,<chr>,<dbl>,<dbl>,<dbl>
BP_Response to Epidermal Growth Factor (GO:0070849),LV2,0.3747462,0.78087597,0.9795112
BP_External Encapsulating Structure Organization (GO:0045229),LV3,0.5169306,0.42070801,0.9006031
BP_Positive Regulation of Adenylate Cyclase Activity (GO:0045762),LV3,0.7231452,0.13033642,0.8514804
BP_Positive Regulation of Blood Vessel Endothelial Cell Migration (GO:0043536),LV3,0.5038280,0.48784783,0.9054424
BP_Positive Regulation of Stress Fiber Assembly (GO:0051496),LV3,0.7549194,0.03459612,0.6071143
BP_Regulation of Mitotic Spindle Assembly (GO:1901673),LV3,0.2968548,0.84711328,0.9795112
BP_Negative Regulation of Cellular Response to Insulin Stimulus (GO:1900077),LV9,0.2422017,0.94979961,0.9795112
BP_Positive Regulation of Stress Fiber Assembly (GO:0051496),LV9,0.4290921,0.69909717,0.9544754
BP_Positive Regulation of Type II Interferon Production (GO:0032729),LV9,0.4414996,0.70097382,0.9544754


In [8]:
head(archs4_summary_sig)

pathway,LV
<chr>,<chr>


In [13]:
archs4_summary_sig_collapsed  <- archs4_summary_sig  %>% 
dplyr::group_by(LV)  %>% 
dplyr::summarise(pathway = paste(pathway, collapse = '; '))

In [14]:
write.csv(archs4_plier2$summary, here('data/archs4/archs4_summary.csv'), row.names = FALSE)

In [15]:
head(archs4_summary_sig_collapsed, n = 20)

LV,pathway
<chr>,<chr>
LV10,CD1C-CD141- Dendritic Cell Blood Human; Microglial Cell Embryonic Prefrontal Cortex Human; Monocyte Fetal Kidney Human; Neutrophil Kidney Human
LV100,Secretory Cell Lung Human
LV1007,Mesangial Cell Kidney Mouse; Podocyte Kidney Mouse
LV1011,Neutrophil Kidney Human
LV1029,Neutrophil Nasal Polyp Human
LV103,DCLK1+ Progenitor Cell Large Intestine Human
LV1043,Goblet Cell Intestinal Crypt Mouse; Transit Amplifying (TA) Cell Intestinal Crypt Mouse
LV1047,Exhausted CD8+ T Cell Liver Human; Regulatory T (Treg) Cell Liver Human; T Cell Brain Human; T Cell Lung Human; T Cell Lymph Human
LV106,Proximal Tubule Cell Kidney Mouse


In [16]:
archs4_summary_trait_sig_collapsed  <- archs4_summary_sig_collapsed  %>% 
dplyr::inner_join(archs4_traits_sig_collapsed, by = 'LV') %>% 
dplyr::arrange(as.numeric(sub('LV', '', LV)))

In [17]:
dim(archs4_summary_trait_sig_collapsed)

[1] 209   3

In [18]:
gtex  <- readRDS(here('output/gtex/gtex_PLIER2.rds'))

In [19]:
write.csv(gtex$summary, here('output/gtex/gtex_summary.csv'), row.names = FALSE)

In [20]:
head(archs4_summary_trait_sig_collapsed)

LV,pathway,phenotype
<chr>,<chr>,<chr>
LV4,Fibroblast Skin Mouse,"6mm weak meridian (left); Systolic blood pressure, automated reading; Exposure to tobacco smoke at home"
LV6,DCLK1+ Progenitor Cell Large Intestine Human; Fibroblast Skin Mouse; Fibrocartilage Chondrocyte Articular Cartilage Human; Leydig Precursor Cell Fetal Gonad Human; Medullary Cell Kidney Mouse; Schwalie Et al.Nature.G2 Adipose Tissue Mouse,6mm weak meridian (right); 3mm strong meridian (right); Hernia; 6mm weak meridian (left); 3mm weak meridian (left); 6mm strong meridian (right); 3mm weak meridian (right); Corneal resistance factor (right); 6mm asymmetry index (right); 6mm strong meridian (left); 3mm strong meridian (left)
LV10,CD1C-CD141- Dendritic Cell Blood Human; Microglial Cell Embryonic Prefrontal Cortex Human; Monocyte Fetal Kidney Human; Neutrophil Kidney Human,Basophill percentage; Basophill count; Monocyte percentage
LV13,Enterocyte Progenitor Cell Small Intestine Human; Hepatocyte Liver Mouse; Proximal Tubule Cell Kidney Mouse,Illnesses of father: Alzheimer's disease/dementia; Job SOC coding: Quality assurance managers; Illnesses of mother: Alzheimer's disease/dementia
LV14,Migration Phase Fetal Germ Cell Fetal Gonad Human; Mitotic Fetal Germ Cell Fetal Gonad Human; Transit Amplifying (TA) Cell Intestinal Crypt Mouse,Illnesses of father: Alzheimer's disease/dementia; Illnesses of mother: Alzheimer's disease/dementia; Illnesses of siblings: Alzheimer's disease/dementia; Any dementia
LV15,Neural Progenitor Cell Embryonic Prefrontal Cortex Human,Difficulty concentrating during worst period of anxiety


# CRISPR CAS9

In [21]:
library(here)
library(dplyr)
library(purrr)
library(fgsea)

In [22]:
lipid_deg <- read.csv(
  here("data/archs4/crispr_cas9/lipid_DEG.csv"),
  stringsAsFactors = FALSE
)

head(lipid_deg)

,gene_name,GFPLow_vs_UnSorted.log2FC,GFPLow_vs_UnSorted.FDR,GFPLow_vs_UnSorted.DEG,GFPHigh_vs_UnSorted.log2FC,GFPHigh_vs_UnSorted.FDR,GFPHigh_vs_UnSorted.DEG,GFPHigh_vs_GFPLow.log2FC,GFPHigh_vs_GFPLow.FDR,GFPHigh_vs_GFPLow.DEG,lipid.effect,rank
,<chr>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<lgl>,<chr>,<int>
1,ABCA2,0.55866839,0.116007580,FALSE,0.78491850,0.02712826,TRUE,0.2262501,0.632690747,FALSE,decrease *,-1
2,ABCB7,0.79852459,0.029219603,TRUE,-0.23834184,0.79077801,FALSE,-1.0368664,0.000693259,TRUE,increase **,2
3,ABCE1,0.68407544,0.047833353,TRUE,0.04432565,0.97525531,FALSE,-0.6397498,0.049228517,TRUE,increase **,2
4,ABCG2,0.43812607,0.049607475,TRUE,0.37554996,0.22953855,FALSE,-0.0625761,0.883677774,FALSE,increase *,1
5,ABHD10,-0.64140454,0.009845648,TRUE,-0.24194344,0.65855335,FALSE,0.3994611,0.156967126,FALSE,decrease *,-1
6,ABHD13,0.08890522,0.806738313,FALSE,-0.45420696,0.04021766,TRUE,-0.5431122,0.000576641,TRUE,increase **,2


In [23]:
orig_deg_gene_sets <- list()
for (r in unique(lipid_deg$rank)) {
  if (r == 0) next
  genes <- lipid_deg$gene_name[lipid_deg$rank == r]
  orig_deg_gene_sets[[paste0("gene_set_", r)]] <- genes
}

head(orig_deg_gene_sets)

$`gene_set_-1`
 [1] "ABCA2"       "ABHD10"      "ADAM32"      "ADAMTS12"    "ALKBH3"     
 [6] "APITD1-CORT" "ASB9"        "ATP5B"       "ATP6AP1"     "C12orf29"   
[11] "C16orf78"    "C21orf140"   "C5orf63"     "CBL"         "CBX1"       
[16] "CDKN1A"      "CETN3"       "COL4A5"      "COPS7A"      "DHRS7"      
[21] "GFRA4"       "GLRA3"       "GNA15"       "GPRASP1"     "GTF2A1L"    
[26] "GUCY2F"      "HELZ2"       "HEXDC"       "HRH2"        "HSBP1L1"    
[31] "IFNA4"       "IQCF2"       "KCNJ5"       "LAMC1"       "LOC645382"  
[36] "LPGAT1"      "MAML1"       "MEDAG"       "NDUFAB1"     "NDUFB8"     
[41] "NLRP3"       "NRIP1"       "OPN1LW"      "PAIP2B"      "PDCL"       
[46] "PDE4B"       "PHYHIP"      "PRR16"       "RPS6KC1"     "RUNX1"      
[51] "S100A5"      "SLAMF8"      "SLC35G3"     "SLC7A3"      "SMIM6"      
[56] "SMPD1"       "SNX13"       "SREBF1"      "STK32C"      "TADA1"      
[61] "TADA2B"      "TAS2R39"     "TCEB1"       "TDRP"        "TMBIM4"     
[66] "TMIGD1"      "TTC38"       "UGT1A5"      "VPS13A"      "VWA1"       
[71] "ZFP36L1"     "ZFP36L2"     "ZNF133"      "ZNF214"      "ZNF583"     
[76] "ZNF7"        "ZNF799"     

$gene_set_2
  [1] "ABCB7"          "ABCE1"          "ABHD13"         "ACTR1A"        
  [5] "ACVR1"          "AGPAT6"         "AKIRIN2"        "AP2S1"         
  [9] "ASCC3"          "ASH2L"          "B3GALT4"        "BDP1"          
 [13] "BMPR1A"         "C1orf109"       "C21orf59"       "C7orf26"       
 [17] "CBLL1"          "CCIN"           "CEBPA"          "CHD4"          
 [21] "CHERP"          "CHP1"           "CPNE6"          "CPSF4"         
 [25] "CRKL"           "CSTF3"          "DCAF7"          "DDTL"          
 [29] "DHX33"          "DKC1"           "DNAJC22"        "DPH1"          
 [33] "DPH3"           "E4F1"           "EIF5"           "EIF6"          
 [37] "ELL"            "ELP4"           "ELP5"           "EMG1"          
 [41] "ERCC3"          "EWSR1"          "EXOC3"          "FBL"           
 [45] "FDX1L"          "FERMT2"         "FNDC3B"         "FNTA"          
 [49] "GATAD2A"        "GNL3L"          "GPN1"           "GPN2"          
 [53] "GTF2B"          "GTF2E2"         "GTF2H1"         "HECTD1"        
 [57] "HNF4A"          "HNRNPH1"        "HNRNPL"         "HSP90B1"       
 [61] "IGF1R"          "IMP4"           "INTS1"          "INTS6"         
 [65] "IPO13"          "IPO7"           "ISY1"           "ITGB1"         
 [69] "KIAA1429"       "KMT2D"          "LSM12"          "LSM4"          
 [73] "LUC7L3"         "MAK16"          "MBTPS2"         "MDM2"          
 [77] "MED8"           "NAPA"           "NARFL"          "NHP2"          
 [81] "NMD3"           "NOL6"           "NOP10"          "NOP16"         
 [85] "NOP2"           "NOP56"          "NR5A1"          "ORAOV1"        
 [89] "OSBP"           "OTUD5"          "PCBP1"          "PDCD2"         
 [93] "PFDN2"          "PFDN6"          "PHF20"          "PLIN2"         
 [97] "POLR2C"         "POLR2F"         "POLR2K"         "POLR2L"        
[101] "POLR3E"         "POP7"           "PPP1R11"        "PWP2"          
[105] "QPCTL"          "RANGAP1"        "RAP1GDS1"       "RBBP6"         
[109] "RPAP3"          "RPL13"          "RPL17-C18orf32" "RPL18"         
[113] "RPL19"          "RPL31"          "RPL34"          "RPL35A"        
[117] "RPL37"          "RPL6"           "RPL7"           "RPL7L1"        
[121] "RPLP0"          "RPP38"          "RPS11"          "RPS13"         
[125] "RPS14"          "RPS16"          "RPS19"          "RPS2"          
[129] "RPS27A"         "RPS28"          "RPS6"           "RPSA"          
[133] "SAFB"           "SAP130"         "SEC61G"         "SLC12A9"       
[137] "SLC33A1"        "SLC35B2"        "SNRPD1"         "SNRPD3"        
[141] "SPATA5"         "SPDYE2"         "SREBF2"         "SRP19"         
[145] "SRP54"          "SRP72"          "STAG2"          "SUDS3"         
[149] "SUPT5H"         "TAF1C"          "TAF6"           "TANGO6"        
[1

In [24]:
deg_gene_sets <- list(
  gene_set_increase = orig_deg_gene_sets[["gene_set_3"]],
  gene_set_decrease = orig_deg_gene_sets[["gene_set_-3"]]
)

head(deg_gene_sets)

$gene_set_increase
[1] "ACACA"  "DGAT2"  "HILPDA" "MBTPS1" "SCAP"   "SRPR"  

$gene_set_decrease
[1] "BLCAP"  "FBXW7"  "INSIG2" "PCYT2"  "PTEN"   "SOX9"   "TCF7L2" "UBE2J2"

In [25]:
lvs <- set_names(
  lapply(seq_len(ncol(archs4_z)), function(i) {
    vec <- archs4_z[, i]
    names(vec) <- rownames(archs4_z)
    vec
  }),
  nm = paste0("LV", seq_len(ncol(archs4_z)))
)

length(lvs)

[1] 2366

In [26]:
n_reps <- 10
set.seed(0)

In [27]:
dir.create(here("output/crispr"), showWarnings = FALSE, recursive = TRUE)
results_file <- here("output/crispr/results.rds")

if (file.exists(results_file)) {
  results <- readRDS(results_file)
} else {
  results <- imap(lvs, ~{
    lv_name <- .y
    reps <- map(seq_len(n_reps), function(i) {
      fg <- fgsea(
        pathways  = deg_gene_sets,
        stats     = .x,
        scoreType = "pos",
        eps       = 0.0
      ) %>%
        arrange(pval) %>%
        mutate(
          lv      = lv_name,
          rep_idx = i
        )
      fg
    })
    bind_rows(reps)
  })
  saveRDS(results, results_file)
}

In [28]:
crispr_cas_results <- bind_rows(results) %>%
  mutate(leadingEdge = map_chr(leadingEdge, toString))

In [29]:
crispr_cas_results_sig  <- crispr_cas_results  %>% 
dplyr::filter(padj < 0.05)

In [30]:
dim(crispr_cas_results_sig)

[1] 411  10

In [31]:
head(crispr_cas_results_sig)

pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
gene_set_decrease,0.01847419,0.03694839,0.3524879,0.9706902,1.219636,8,"SOX9, INSIG2",LV4,1
gene_set_decrease,0.02358078,0.04716156,0.3524879,0.9706902,1.235971,8,"SOX9, INSIG2",LV4,2
gene_set_decrease,0.02418156,0.04836311,0.3524879,0.9706902,1.224327,8,"SOX9, INSIG2",LV4,3
gene_set_decrease,0.01516995,0.03033989,0.3807304,0.9706902,1.216527,8,"SOX9, INSIG2",LV4,4
gene_set_decrease,0.02497502,0.04995005,0.2878571,0.9706902,1.234782,8,"SOX9, INSIG2",LV4,5
gene_set_decrease,0.02237923,0.04475846,0.3524879,0.9706902,1.235104,8,"SOX9, INSIG2",LV4,8


In [32]:
crispr_cas_results_sig_sub  <- crispr_cas_results_sig  %>% 
dplyr::rename(LV=lv)  %>% 
dplyr::rename(pathway_gene=pathway)  %>% 
dplyr::select(LV, pathway_gene, leadingEdge)

In [33]:
crispr_cas_results_sig_sub  %>% 
dplyr::inner_join(archs4_traits_sig_collapsed)  %>% 
unique()

Joining with `by = join_by(LV)`


LV     pathway_gene      leadingEdge                 
1  LV4    gene_set_decrease SOX9, INSIG2                
2  LV37   gene_set_decrease PTEN, BLCAP                 
3  LV38   gene_set_increase HILPDA, ACACA               
4  LV53   gene_set_increase MBTPS1, ACACA, HILPDA       
5  LV100  gene_set_decrease SOX9, BLCAP                 
6  LV165  gene_set_decrease PCYT2, BLCAP                
7  LV239  gene_set_increase DGAT2, MBTPS1               
8  LV255  gene_set_decrease FBXW7, SOX9, BLCAP          
9  LV255  gene_set_increase DGAT2, ACACA                
10 LV282  gene_set_increase DGAT2, ACACA                
11 LV339  gene_set_decrease PTEN, TCF7L2, BLCAP         
12 LV379  gene_set_decrease PTEN, BLCAP                 
13 LV442  gene_set_increase HILPDA, ACACA               
14 LV455  gene_set_increase HILPDA, ACACA               
15 LV469  gene_set_increase ACACA, SCAP                 
16 LV512  gene_set_decrease SOX9, BLCAP                 
17 LV718  gene_set_increase HILPDA, ACACA               
18 LV773  gene_set_increase DGAT2, ACACA                
19 LV836  gene_set_decrease SOX9, PCYT2                 
20 LV843  gene_set_decrease TCF7L2, INSIG2, BLCAP       
21 LV897  gene_set_increase SCAP, ACACA                 
22 LV1039 gene_set_decrease SOX9, TCF7L2                
23 LV1099 gene_set_increase DGAT2, SCAP                 
24 LV1112 gene_set_increase HILPDA, ACACA               
25 LV1209 gene_set_increase HILPDA, SCAP                
26 LV1209 gene_set_decrease SOX9, UBE2J2                
27 LV1249 gene_set_increase DGAT2, ACACA                
28 LV1307 gene_set_increase HILPDA, ACACA               
29 LV1423 gene_set_increase DGAT2, HILPDA               
30 LV1430 gene_set_increase DGAT2, ACACA                
31 LV1464 gene_set_increase DGAT2, ACACA                
32 LV1642 gene_set_decrease SOX9, PTEN, BLCAP           
33 LV1671 gene_set_decrease SOX9, UBE2J2                
34 LV1704 gene_set_increase HILPDA, ACACA               
35 LV1919 gene_set_increase HILPDA, ACACA               
36 LV1954 gene_set_decrease SOX9, INSIG2, FBXW7         
37 LV1972 gene_set_increase DGAT2, ACACA                
38 LV1981 gene_set_increase DGAT2, ACACA                
39 LV1997 gene_set_decrease SOX9, TCF7L2, INSIG2        
40 LV2007 gene_set_increase HILPDA, ACACA               
41 LV2017 gene_set_decrease UBE2J2, TCF7L2              
42 LV2032 gene_set_increase DGAT2, MBTPS1               
43 LV2050 gene_set_decrease TCF7L2, FBXW7, SOX9         
44 LV2076 gene_set_decrease SOX9, BLCAP                 
45 LV2224 gene_set_decrease BLCAP, FBXW7                
46 LV2258 gene_set_increase HILPDA, ACACA               
47 LV2264 gene_set_increase DGAT2, ACACA                
48 LV2301 gene_set_decrease TCF7L2, INSIG2, SOX9, UBE2J2
   phenotype                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [34]:
crispr_cas_results_sig_sub  %>% 
dplyr::inner_join(archs4_traits_sig_collapsed)  %>% 
unique()  %>% 
pull(phenotype)  %>% 
strsplit('; ')  %>%
unlist()  %>% 
unique()

Joining with `by = join_by(LV)`


[1] "6mm weak meridian (left)"                                                                                                                                                                                              
  [2] "Systolic blood pressure, automated reading"                                                                                                                                                                            
  [3] "Exposure to tobacco smoke at home"                                                                                                                                                                                     
  [4] "Diagnoses - main ICD10: L29 Pruritus"                                                                                                                                                                                  
  [5] "Past tobacco smoking"                                                                                                                                                                                                  
  [6] "Smoking status: Never"                                                                                                                                                                                                 
  [7] "Ever smoked"                                                                                                                                                                                                           
  [8] "Tobacco smoking: Ex-smoker"                                                                                                                                                                                            
  [9] "Diagnoses - main ICD10: J46 Status asthmaticus"                                                                                                                                                                        
 [10] "Asthma (mode)"                                                                                                                                                                                                         
 [11] "Treatment/medication code: terazosin (20003_1140879798)"                                                                                                                                                               
 [12] "Treatment/medication code: prednisone (20003_1140868364)"                                                                                                                                                              
 [13] "Illnesses of siblings: Breast cancer"                                                                                                                                                                                  
 [14] "Diagnoses - main ICD10: C50 Malignant neoplasm of breast"                                                                                                                                                              
 [15] "Malignant neoplasm of breast"                                                                                                                                                                                          
 [16] "Non-cancer illness code, self-reported: high cholesterol"                                                                                                                                                              
 [17] "Illnesses of father: Alzheimer's disease/dementia"                                                                                                                                                                     
 [18] "Medication for cholesterol, blood pressure, diabetes, or take exogenous hormones: Cholesterol lowering medication"                                                                                          

In [35]:
archs4_plier2$summary %>% 
dplyr::rename(LV = `LV index`)  %>% 
dplyr::mutate(LV = paste0('LV', LV))  %>% 
dplyr::filter(LV %in% c('LV165'))

pathway,LV,AUC,p-value,FDR
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Microglial Cell Brain Mouse,LV165,0.6226083,1.486604e-08,4.442291e-07
Microglial Cell Embryonic Prefrontal Cortex Human,LV165,0.6167741,3.018840e-06,5.498717e-05
Mitotic Fetal Germ Cell Fetal Gonad Human,LV165,0.5439155,2.387380e-02,8.637614e-02
Oogenesis Phase Fetal Germ Cell Fetal Gonad Human,LV165,0.5373284,3.422353e-02,1.116156e-01
Regulatory T (Treg) Cell Liver Human,LV165,0.5376881,8.621588e-02,2.089622e-01
Type I Spiral Ganglion Neuron Brain Mouse,LV165,0.5522254,1.787807e-03,1.275645e-02


In [42]:
archs4_z  %>% 
dplyr::select(LV165)  %>%
dplyr::arrange(desc(LV165))  %>%
head(n = ceiling(0.01 * nrow(.))) %>% 
rownames() %>% 
paste(collapse = '\n') %>% 
cat()


RRAGC-DT
UAP1L1
RENBP
HSD17B14
RAB3IL1
RRAGD
GNPDA1
HS1BP3
AMDHD2
FLCN
GPNMB
MCOLN3
HMOX1
NEU1
CLCN5
SNX8
TPP1
CTSA
CCDC154
SH2D5
PLD6
CCN3
NCR3LG1
FNIP1
GLMP
SLC38A7
STX3
HEXA
BRI3
RRAGC
GOLGA4-AS1
FNIP2
DDIT3
PIFO
PFKFB2
NEURL2
SQSTM1
RAB29
AP5Z1
LBX2-AS1
ZFYVE26
WASHC2A
GDF15
CIART
CTNS
CELF6
RPP25
SMIM29
MCOLN1
THAP8
ASAH1
NPC2
CLCN7
CPEB1
ATP6V1C1
PEPD
NUDT14
PLIN2
CTSD
WASHC2C
SLC17A5
GNS
OSTM1
RAB32
SCPEP1
MANBA
NEURL3
FOXRED2
LY96
SLC66A1
HRK
PTGDS
WDR81
PNMA6A
NAGLU
SAT2
FAM21FP
OSGIN1
TMEM251
DHDH
MROH1
MIR616
DNASE2
PKD1L2
VEGFB
FOLR3
IFI30
HSPBAP1
GRN
KLHL24
RHEBL1
PPP1R3E
BLVRB
DPP7
GBA
TMEM192
VPS18
TNFRSF14-AS1
ARMCX1
CTSL
PLA2G4C
SHC2
SPNS1
HEXB
ATP6V1B2
CEMP1
APOE
DUSP3
LAMP2
KDM2B-DT
CLN3
VAT1
HPS1
SPG21
GEM
TTC21A
WDSUB1
FTL
IL6R
SPATA25
LACTB2
PIP4P1
SLC12A7
SLC26A11
GBAP1
MTHFR
MCTP1
SNX16
VPS26A
P2RX4
FAM21EP
NUPR1
GALK2
CYB5RL
KRT8P46
IGF2R
TMEM38B
ATP6AP1-DT
PPARGC1A
CLTA
FBXO10
HKDC1
CLCN6
TMEM144
MAFG
CRACD
RCBTB1
PPM1H
FABP3
UCN
SLC36A1
KLHL21
SPRING1
LINC005

In [37]:
archs4_z  %>% 
dplyr::select(LV165)  %>% 
head()

,LV165
,<dbl>
A1BG,0.0000000
A1BG-AS1,0.0571732
A2M,0.0000000
A2M-AS1,0.0000000
A2ML1,0.2789111
A4GALT,0.0000000
